# WARC Index Status

We can use the Tracking Database check the WARC index status.

The CDX index status can be broken down by day using the following facet query (taking advantage of [Solr's JSON Facet API](https://lucene.apache.org/solr/guide/8_4/json-facet-api.html)).

In [9]:
import json
import requests

headers = {'content-type': "application/json" }

json_facet = {
    # Primary facet is by date - here we break down the last month(s) into days
    'facet': {
        'dates' : { 
            'type' : 'range', 
            'field' : 'timestamp_dt', 
            'start' : "NOW/DAY-6MONTH",
            'end' : "NOW/DAY", 
            'gap' : "+1DAY", 
            # For each day, we facet based on the CDX Index field, and make sure items with no value get recorded:
            'facet': { 
                'index_status': { 
                    'terms': { 
                        "field": "cdx_index_ss", 
                        'missing': True 
                    }
                }
            }
        } 
    }
}


params = {
  'q': 'kind_s:"warcs"',
  'rows': 0
}

r = requests.post("http://solr8.api.wa.bl.uk/solr/tracking/select", params=params, data=json.dumps(json_facet), headers=headers)

if r.status_code != 200:
    print(r.text)

from solr_facet_helper import flatten_solr_buckets

df = pd.DataFrame(flatten_solr_buckets(r.json()['facets']))
# Filter empty rows:
df=df[df['count'] != 0]

df

,dates,index_status,count
0,2020-01-17T00:00:00Z,data-heritrix,214
1,2020-01-17T00:00:00Z,missing,2
2,2020-01-18T00:00:00Z,data-heritrix,203
3,2020-01-18T00:00:00Z,missing,19
4,2020-01-19T00:00:00Z,data-heritrix,231
...,...,...,...
353,2020-07-12T00:00:00Z,data-heritrix,104
355,2020-07-13T00:00:00Z,data-heritrix,80
357,2020-07-14T00:00:00Z,data-heritrix,118
359,2020-07-15T00:00:00Z,data-heritrix,97


Which can be used to build a simple visualisation:

In [13]:
import altair as alt

alt.Chart(df).mark_bar(size=6).encode(
    x='dates:T',
    y='count',
    color='index_status',
    tooltip=[alt.Tooltip('dates:T', format='%A, %e %B %Y'),'index_status', 'count']
).properties(width=600).interactive()

alt.Chart(...)